#Classification des document par catégories

#Imports des librairies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Installing the transformers library and additional libraries if looking process 

!pip install -q transformers

In [ ]:
# Importing stock ml libraries
import pickle
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig
from sklearn.model_selection import train_test_split
tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.1') #on essaie aussi 'allenai/scibert_scivocab_uncased', 'gsarti/covidbert-nli' en plus du traditionnel 'bert-base-uncased', et tous ont de moins bons résutats que BioBert
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


#Import des données

On effectue également un traitement sur notre DataFrame de référence

In [ ]:
if True:
  df_bibliovid=pd.read_json('/content/drive/MyDrive/data/bibliovid.json')
  df_cord=pd.read_json('/content/drive/MyDrive/data/cord19-metadata.json')
  df_litcovid=pd.read_json('/content/drive/MyDrive/data/litcovid.json')
  df_litcovid=df_litcovid.dropna(subset=['abstract','topics'])
  df_litcovid =df_litcovid.drop(df_litcovid[ df_litcovid.abstract==''].index)
  df_litcovid =df_litcovid.drop(df_litcovid[ df_litcovid.topics==''].index)

In [ ]:
def vectorize(label_list,dico):
    return [int(elt in label_list) for  elt in dico]

In [ ]:
if True:
  topic_dict=dict()
  topic_list=[]
  for elt in df_litcovid['topics'].values:
      topic_list+=elt
  print(topic_list)
  topic_set=set(topic_list)
  i=0
  for elt in topic_set:
    topic_dict[elt]=i
    i+=1
  topic_dict
  df_litcovid['vect_topics']=df_litcovid.topics.apply(lambda x: vectorize(x,topic_dict))
  df_litcovid['cat_text']=df_litcovid['title']+df_litcovid['abstract']
  df_litcovid['len']=df_litcovid['cat_text'].apply(lambda x: len(x.split(' ')))
  df_litcovid['vect_topics']
  df_litcovid=df_litcovid.reset_index(drop=True)

['Mechanism', 'Treatment', 'Prevention', 'Treatment', 'Prevention', 'Prevention', 'Prevention', 'Prevention', 'Prevention', 'Treatment', 'Prevention', 'Mechanism', 'Treatment', 'Diagnosis', 'Diagnosis', 'Treatment', 'Diagnosis', 'Treatment', 'Diagnosis', 'Prevention', 'Prevention', 'Treatment', 'Treatment', 'Diagnosis', 'Treatment', 'Mechanism', 'Treatment', 'Prevention', 'Mechanism', 'Treatment', 'Diagnosis', 'Treatment', 'Mechanism', 'Prevention', 'Diagnosis', 'Treatment', 'Prevention', 'Prevention', 'Treatment', 'Case Report', 'Diagnosis', 'Case Report', 'Diagnosis', 'Diagnosis', 'Treatment', 'Prevention', 'Diagnosis', 'Treatment', 'Diagnosis', 'Prevention', 'Diagnosis', 'Mechanism', 'Treatment', 'Mechanism', 'Diagnosis', 'Diagnosis', 'Diagnosis', 'Treatment', 'Prevention', 'Prevention', 'Prevention', 'Diagnosis', 'Diagnosis', 'Treatment', 'Diagnosis', 'Treatment', 'Diagnosis', 'Diagnosis', 'Treatment', 'Prevention', 'Diagnosis', 'Treatment', 'Prevention', 'Treatment', 'Prevention',

In [ ]:
input_ids=[]
attention_mask=[]
token_type_ids=[]
def make_dict(l):
    d=dict()
    for elt in l:
      d[elt]=''
    return d
def remove_ponctuation(text,ponctuation):
    return ''.join([elt for elt in list(text) if elt not in ponctuation])
def remove_stopword(text):
    return ' '.join([word for word in nltk.word_tokenize((text.lower())) if word not in make_dict(stopwords.words('english'))])
if True:
  ponctuation='''!()-[]{};:'"\,<>./?@#$%^&*_~'''
  with  tqdm(total=df_litcovid.shape[0], position=0) as progress:
            
    for index, row in df_litcovid.iterrows(): #tqdm((df_litcovid.iterrows()),total=df_litcovid.shape[0],initial=0):
      x=tokenizer.encode_plus(
                remove_stopword(remove_ponctuation(row.cat_text,ponctuation)),
                add_special_tokens=True,
                max_length=200,
                pad_to_max_length=True,
                padding=False,
                return_token_type_ids=True
            )
      
      input_ids.append(x['input_ids'])
      attention_mask.append(x['attention_mask'])
      token_type_ids.append(x['token_type_ids'])
      progress.update()

  0%|          | 0/41917 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 41917/41917 [20:13<00:00, 34.54it/s]


In [ ]:
if True:
  a=df_litcovid.copy()
  a['input_ids']=input_ids
  a['attention_mask']=attention_mask
  a['token_type_ids']=token_type_ids
  #a.head()

In [ ]:
a

,pmid,title,journal,authors,date,_id,topics,text_hl,abstract,countries,pmcid,vect_topics,cat_text,len,input_ids,attention_mask,token_type_ids
0,33341814,Hyperinflammation and Immune Response Generati...,Neuroimmunomodulation,"[Mishra, Kamla Prasad, Singh, Ajay Kumar, Sing...",2020-12-21 12:00:00+00:00,33341814,"[Mechanism, Treatment]",NaN,Severe acute respiratory syndrome coronavirus-...,NaN,NaN,"[1, 0, 0, 0, 0, 0, 0, 1, 0]",Hyperinflammation and Immune Response Generati...,117,"[102, 1884, 6652, 150, 3890, 1278, 3014, 21004...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,33341217,COVID-19 pandemic and the skin: what should de...,Clin Dermatol,"[Darlenski, Razvigor, Tsankov, Nikolai]",2020-12-21 12:00:00+00:00,33341217,[Prevention],NaN,The world has changed dramatically since the C...,NaN,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 1]",COVID-19 pandemic and the skin: what should de...,110,"[102, 21004, 173, 1685, 26196, 3843, 25777, 24...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,33341119,Coronavirus vaccine development: from SARS and...,J Biomed Sci,"[Li, Yen-Der, Chi, Wei-Yu, Su, Jun-Han, Ferral...",2020-12-21 12:00:00+00:00,33341119,[Treatment],NaN,Severe Acute Respiratory Syndrome Coronavirus ...,NaN,NaN,"[1, 0, 0, 0, 0, 0, 0, 0, 0]",Coronavirus vaccine development: from SARS and...,147,"[102, 28864, 12833, 7039, 1120, 6460, 30113, 5...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,33341242,Recommendations on intervention for hepatobili...,Cir Esp,"[Garcia Botella, Alejandra, Gomez Bravo, Migue...",2020-12-21 12:00:00+00:00,33341242,[Prevention],NaN,The SARS-CoV-2 (COVID-19) pandemic requires an...,NaN,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 1]",Recommendations on intervention for hepatobili...,144,"[102, 6869, 3832, 6188, 9739, 426, 1212, 13097...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,33341205,Management strategies for dermatomyositis duri...,Clin Dermatol,"[Pi, Zixin, Chen, Pan, Zhan, Yi, Xiao, Rong]",2020-12-21 12:00:00+00:00,33341205,[Prevention],NaN,"In late 2019, the coronavirus disease 2019 (CO...",[China],NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 1]",Management strategies for dermatomyositis duri...,94,"[102, 1837, 3236, 13460, 1796, 27391, 28864, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41912,31967321,Cross-species transmission of the newly identi...,J Med Virol,"[Ji, Wei, Wang, Wei, Zhao, Xiaofang, Zai, Junj...",2020-01-23 12:00:00+00:00,31967321,"[Transmission, Mechanism]",NaN,The current outbreak of viral pneumonia in the...,[China],PMC7138088,"[0, 1, 0, 0, 0, 0, 0, 1, 0]",Cross-species transmission of the newly identi...,175,"[102, 2057, 15683, 2856, 7737, 1887, 28864, 12...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
41913,31967327,"Emerging coronaviruses: Genome structure, repl...",J Med Virol,"[Chen, Yu, Liu, Qianyun, Guo, Deyin]",2020-01-23 12:00:00+00:00,31967327,[Mechanism],NaN,The recent emergence of a novel coronavirus (2...,[China],PMC7167049,"[0, 0, 0, 0, 0, 0, 0, 1, 0]","Emerging coronaviruses: Genome structure, repl...",90,"[102, 7602, 28864, 26700, 4165, 1187, 6436, 86...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
41914,31943059,"Pneumonia of unknown aetiology in Wuhan, China...",J Travel Med,"[Bogoch, Isaac I, Watts, Alexander, Thomas-Bac...",2020-01-17 12:00:00+00:00,31943059,[Transmission],NaN,There is currently an outbreak of pneumonia of...,[China],PMC7107534,"[0, 1, 0, 0, 0, 0, 0, 0, 0]","Pneumonia of unknown aetiology in Wuhan, China...",55,"[102, 9520, 4384, 24073, 665, 7867, 6870, 3640...","[1, 1, 1, 1, 1, 1, 1, 

Configuration

In [ ]:
MAX_LEN = 200
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 1
LEARNING_RATE = 3e-5 #conseillé sur le papier
tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.1') #on essaie aussi 'allenai/scibert_scivocab_uncased', 'gsarti/covidbert-nli' en plus du traditionnel 'bert-base-uncased', et tous ont de moins bons résutats que BioBert

Classe Dataset

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, dataframe):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.comment_text = dataframe.abstract #ajouter le titre
        self.targets = self.data.vect_topics
        self.input_ids=[]
        self.attention_mask=[]
        self.token_type_ids=[]
    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, index):
        self.input_ids=self.data.input_ids.iloc[index]
        self.attention_mask=self.data.attention_mask.iloc[index]
        self.token_type_ids=self.data.token_type_ids.iloc[index]
        return {
            'ids': torch.tensor(self.input_ids, dtype=torch.long),
            'mask': torch.tensor(self.attention_mask, dtype=torch.long),
            'token_type_ids': torch.tensor(self.token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

Datasets+Loaders

In [ ]:
train_size = 0.8
#train_dataset=df_litcovid.sample(frac=train_size,random_state=200)
#test_dataset=df_litcovid.drop(train_dataset.index).reset_index(drop=True)
train_dataset=a.sample(frac=train_size,random_state=200)
test_dataset=a.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


#print("FULL Dataset: {}".format(df_litcovid.shape))
print("FULL Dataset: {}".format(a.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset)
testing_set = CustomDataset(test_dataset)

FULL Dataset: (41917, 17)
TRAIN Dataset: (33534, 17)
TEST Dataset: (8383, 17)


In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

#Création d'un nouveau modèle

On crée un nouveau modèle à partir de la version BioBert de Bert

In [ ]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('dmis-lab/biobert-base-cased-v1.1')
        #for param in self.l1.parameters():
        #  param.requires_grad = False
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 9)
    
    def forward(self, ids, mask, token_type_ids):
        output1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        #output_2 = self.l2(output1[1])
        output = self.l3(output1[1])
        return output

if True:
  model = BERTClass()
  model.to(device)

In [ ]:
def model_parameters_number(model):
  l=[]
  for parameter in model.parameters():
      l.append(parameter)
  s=0
  for el in l:
    try :s+=el.shape[0]*el.shape[1]
    except: s+=el.shape[0]
  return s
model_parameters_number(model)

108317193

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

#Entrainement

In [ ]:
def train(epochs):
    model.train()
    for epoch in range(epochs):
        for _,data in tqdm(enumerate(training_loader),total=len(training_loader),position=0,leave=True):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)

            outputs = model(ids, mask, token_type_ids)

            optimizer.zero_grad()
            loss = loss_fn(outputs, targets)
            if _%len(training_loader)==0:
                print(f'Epoch: {epoch}, Loss:  {loss.item()}')
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()    

On reprend les mêmes conditions expérimentales pour toutes les versions de Bert utilisées, ce qui nous permet ensuite de comparer les résultats obtenus

In [ ]:
train(3)#Bert with no dropout

  0%|          | 0/1048 [00:00<?, ?it/s]

Epoch: 0, Loss:  0.718258261680603


  0%|          | 0/1048 [00:00<?, ?it/s]

Epoch: 1, Loss:  0.09783633798360825


  0%|          | 0/1048 [00:00<?, ?it/s]

Epoch: 2, Loss:  0.03614049032330513


100%|██████████| 1048/1048 [20:07<00:00,  1.15s/it]


#Validation du modèle:

On utilise les metriques: 
- Accuracy Score
- F1 Micro
- F1 Macro


In [ ]:
def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader),total=len(testing_loader),position=0,leave=True):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
for epoch in range(EPOCHS):
    outputs, targets = validation(epoch)
    outputs = np.array(outputs) >= 0.5
    accuracy = metrics.accuracy_score(targets, outputs)
    f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
    f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
    print(f"Accuracy Score = {accuracy}")
    print(f"F1 Score (Micro) = {f1_score_micro}")
    print(f"F1 Score (Macro) = {f1_score_macro}")

100%|██████████| 262/262 [01:45<00:00,  2.49it/s]

Accuracy Score = 0.7894548490993678
F1 Score (Micro) = 0.8972690694289872
F1 Score (Macro) = 0.7382833817948219


On obtient avec BioBert les meilleurs résultats en accuracy, F1 Score (Micro) et (Macro) de tous nos modèles utilisés